# MSADS 508 CyberSentinel Security Solutions
## Athena Database Schema

In [1]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [2]:
ingest_create_athena_db_passed = False

### Check Pre-Requisites

In [3]:
%store -r s3_public_csv

In [4]:
try:
    s3_public_csv
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

In [5]:
print(s3_public_csv)

s3://msads-508-sp25-team6


In [6]:
%store -r s3_private_csv

In [7]:
try:
    s3_private_csv
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

In [8]:
print(s3_private_csv)

s3://sagemaker-us-east-1-367086635748/msads-508-sp25-team6


### Import PyAthena

In [9]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

## Create Athena Database

In [10]:
database_name = "cybersentinel"

In [11]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [12]:
conn = connect(region_name = region, s3_staging_dir = s3_staging_dir)

In [13]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS cybersentinel


In [14]:
import pandas as pd
pd.read_sql(statement, conn)

/tmp/ipykernel_1433/1322682284.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


### Verify The Database Has Been Created

In [15]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_1433/3999478089.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,cybersentinel
1,default


In [16]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

In [17]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


### Store Variables

In [18]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed             -> True
s3_cybersentinel_csv                       -> 's3://msads-508-sp25-team6/MSADS 508 Final Project
s3_private_csv                             -> 's3://sagemaker-us-east-1-367086635748/msads-508-s
s3_public_csv                              -> 's3://msads-508-sp25-team6'
setup_dependencies_passed                  -> True
setup_iam_roles_passed                     -> True
setup_instance_check_passed                -> True
setup_s3_bucket_passed                     -> True


## Register CSV Data with Athena

In [19]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [20]:
ingest_create_athena_table_csv_passed = False

In [21]:
%store -r ingest_create_athena_db_passed

In [22]:
try:
    ingest_create_athena_db_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [23]:
print(ingest_create_athena_db_passed)

True


In [24]:
if not ingest_create_athena_db_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [25]:
%store -r s3_private_csv

In [26]:
try:
    s3_private_csv
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

In [27]:
print(s3_private_csv)

s3://sagemaker-us-east-1-367086635748/msads-508-sp25-team6


### Import PyAthena

In [28]:
from pyathena import connect

### Create Athena Table from Local CSV File

In [29]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [38]:
# Set Athena parameters
database_name = "cybersentinel"
table_name_csv = "cyber_sentinel_security_csv"

In [39]:
conn = connect(region_name = region, s3_staging_dir = s3_staging_dir)

In [42]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
            source_ip string,
            source_port string,
            destination_ip string,
            destination_port string,
            protocol string,
            flow_duration float,
            total_fwd_packets int,
            total_backward_packets int,
            fwd_packet_length_mean float,
            bwd_packet_length_mean float,
            label string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\\,' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count' = '1')""".format(
    database_name, table_name_csv, s3_private_csv
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS cybersentinel.cyber_sentinel_security_csv(
            source_ip string,
            source_port string,
            destination_ip string,
            destination_port string,
            protocol string,
            flow_duration float,
            total_fwd_packets int,
            total_backward_packets int,
            fwd_packet_length_mean float,
            bwd_packet_length_mean float,
            label string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-367086635748/msads-508-sp25-team6'
TBLPROPERTIES ('skip.header.line.count' = '1')


In [43]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_1433/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


### Verify The Table Has Been Created

In [44]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_1433/2201015668.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,cyber_sentinel_security_csv


In [45]:
if table_name_csv in df_show.values:
    ingest_create_athena_table_csv_passed = True

In [46]:
%store ingest_create_athena_table_csv_passed

Stored 'ingest_create_athena_table_csv_passed' (bool)


### Run A Sample Query

In [52]:
label = "benign"

statement = """SELECT * FROM {}.{}
        WHERE label = '{}' LIMIT 100""".format(
            database_name, table_name_csv, label
        )

print(statement)

SELECT * FROM cybersentinel.cyber_sentinel_security_csv
        WHERE label = 'benign' LIMIT 100


In [53]:
df = pd.read_sql(statement, conn)

df.head(5)

/tmp/ipykernel_1433/2219640993.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,source_ip,source_port,destination_ip,destination_port,protocol,flow_duration,total_fwd_packets,total_backward_packets,fwd_packet_length_mean,bwd_packet_length_mean,label
0,59.166.0.5,10475,149.171.126.8,44054,tcp,24.838,46,46,62.0,669.0,benign
1,59.166.0.7,19565,149.171.126.2,21,tcp,923.815,52,54,56.0,69.0,benign
2,59.166.0.3,46695,149.171.126.8,52783,tcp,24.590,50,50,62.0,624.0,benign
3,59.166.0.7,60266,149.171.126.2,61745,tcp,223.960,14,6,638.0,53.0,benign
4,59.166.0.3,53177,149.171.126.2,56996,tcp,52.046,90,92,57.0,936.0,benign


### Store Variables

In [54]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed                    -> True
ingest_create_athena_table_csv_passed             -> True
s3_cybersentinel_csv                              -> 's3://msads-508-sp25-team6/MSADS 508 Final Project
s3_private_csv                                    -> 's3://sagemaker-us-east-1-367086635748/msads-508-s
s3_public_csv                                     -> 's3://msads-508-sp25-team6'
setup_dependencies_passed                         -> True
setup_iam_roles_passed                            -> True
setup_instance_check_passed                       -> True
setup_s3_bucket_passed                            -> True
